<a href="https://colab.research.google.com/github/irham235/Belajar-machine-learning/blob/main/multiclass_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install kaggle

In [6]:
from google.colab import files

# upload kaggle.json
uploaded = files.upload()

Saving kaggle (4).json to kaggle (4).json


In [7]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [8]:
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d 'antoniuscs/imdb-synopsis-indonesian-movies'

  0% 0.00/299k [00:00<?, ?B/s]
100% 299k/299k [00:00<00:00, 124MB/s]


In [10]:
import zipfile
dataset_zip = zipfile.ZipFile('imdb-synopsis-indonesian-movies.zip', 'r')
dataset_zip.extractall()
dataset_zip.close()

Pada cell pertama impor library pandas dan ubah dataset menjadi dataframe. Kemudian buang kolom 'judul_film' karena kita hanya akan menggunakan sinopsis sebagai atribut untuk dilatih pada model.

In [11]:
import pandas as pd
df = pd.read_csv('/content/imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])

Panggil fungsi head() pada dataframe untuk menampilkan 5 sampel teratas pada dataset.

In [12]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


Karena label kita berupa data kategorikal, maka kita perlu melakukan proses one-hot-encoding. kode di bawah untuk melakukan one-hot-encoding dan membuat dataframe baru.

In [13]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


Agar dapat diproses oleh model, kita perlu mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values.

In [14]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

Lalu, bagi data untuk training dan data untuk testing.

In [15]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

Kemudian kita ubah setiap kata pada dataset kita ke dalam bilangan numerik dengan fungsi Tokenizer. Setelah tokenisasi selesai, kita perlu membuat mengonversi setiap sampel menjadi sequence.

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih)
tokenizer.fit_on_texts(sinopsis_test)

sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

Untuk arsitektur model kita menggunakan layer Embedding dengan dimensi embedding sebesar 16, serta dimensi dari input sebesar nilai num_words pada objek tokenizer. Jangan lupa panggil fungsi compile dan tentukan optimizer serta loss function yang akan dipakai oleh model.

In [17]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Terakhir kita dapat mulai melatih model kita dengan memanggil fungsi fit().

In [18]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs,
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 10s - loss: 1.6104 - accuracy: 0.1679 - val_loss: 1.6109 - val_accuracy: 0.1393 - 10s/epoch - 383ms/step
Epoch 2/30
26/26 - 5s - loss: 1.6086 - accuracy: 0.2214 - val_loss: 1.6129 - val_accuracy: 0.1343 - 5s/epoch - 202ms/step
Epoch 3/30
26/26 - 4s - loss: 1.5933 - accuracy: 0.2799 - val_loss: 1.6728 - val_accuracy: 0.1343 - 4s/epoch - 162ms/step
Epoch 4/30
26/26 - 5s - loss: 1.4777 - accuracy: 0.4017 - val_loss: 1.6158 - val_accuracy: 0.2338 - 5s/epoch - 207ms/step
Epoch 5/30
26/26 - 4s - loss: 1.0727 - accuracy: 0.5871 - val_loss: 1.6675 - val_accuracy: 0.2935 - 4s/epoch - 170ms/step
Epoch 6/30
26/26 - 5s - loss: 0.6289 - accuracy: 0.7351 - val_loss: 2.2132 - val_accuracy: 0.2836 - 5s/epoch - 178ms/step
Epoch 7/30
26/26 - 4s - loss: 0.3919 - accuracy: 0.8632 - val_loss: 2.4946 - val_accuracy: 0.3184 - 4s/epoch - 161ms/step
Epoch 8/30
26/26 - 3s - loss: 0.1858 - accuracy: 0.9540 - val_loss: 3.0807 - val_accuracy: 0.3483 - 3s/epoch - 132ms/step
Epoch 9/30
26/26 - 3s 

Akurasi dari model kita menunjukkan terjadinya overfitting karena akurasi pada data training sangat besar, sedangkan akurasi pada data validasi jauh lebih kecil. Hal ini masih sangat luar biasa karena kita hanya memiliki 1000 buah sampel data!